In [ ]:
## Name of both participiants: Ďuriš Maroš, Fedor Richard

In [ ]:
from PIL import Image, ImageOps
from google.colab import files
import io
import IPython.display as display
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
import numpy as np
import os

MODEL_PATH = "mnist_model.h5"  # Path to save/load the model

def load_and_train_model():
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Preprocess data
    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

    # Build the model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

    # Save the model after training
    model.save(MODEL_PATH)
    print(f"Model trained and saved to {MODEL_PATH}")

    return model

def process_and_display_image(model):
    # Prompt the user to upload an image
    print("Please upload an image file:")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return

    try:
        # Load the uploaded image
        file_name = next(iter(uploaded))
        image = Image.open(io.BytesIO(uploaded[file_name]))

        # Convert the image to grayscale
        grayscale_image = ImageOps.grayscale(image)

        # Resize to 28x28 pixels
        resized_image = grayscale_image.resize((28, 28))

        # Invert the colors of the image
        inverted_image = ImageOps.invert(resized_image)

        # Normalize the image to match model input
        image_array = np.array(inverted_image).astype('float32') / 255.0
        image_array = image_array.reshape(1, 28, 28, 1)

        # Display the processed image
        print("Processed Image (28x28 grayscale, inverted):")
        display.display(inverted_image)

        # Predict the digit using the model
        prediction = model.predict(image_array)
        predicted_digit = np.argmax(prediction)
        print(f"Predicted Digit: {predicted_digit}")

    except Exception as e:
        print(f"Error processing the image: {e}")

# Check if the model already exists and load it if possible
if os.path.exists(MODEL_PATH):
    print(f"Loading existing model from {MODEL_PATH}")
    model = tf.keras.models.load_model(MODEL_PATH)
else:
    print("No model found. Training a new model...")
    model = load_and_train_model()

# Run the function to process the image
process_and_display_image(model)


In [27]:
process_and_display_image(model)

Please upload an image file:


Saving 8 (1).png to 8 (1).png
Processed Image (28x28 grayscale, inverted):


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Digit: 6
